## "서울시 버스노선별 정류장별 승하차 인원 정보 데이터" 전처리 과정

## 데이터 전처리 실습을 통해 해결해야 할 질문들

- 실습문제 1. 데이터 로딩하기
 
- 실습문제 2. 기본정보 확인하기

- 실습문제 3. 버스정류장 위치를 구 별로 구분하기

- 실습문제 4. (가상)버스정류장 제거하기

- 실습문제 5. 구 별로 버스정류장이 몇 개 있을까?

- 실습문제 6. 구 별로 버스 노선이 몇 개나 지나갈까?

- 실습문제 7. 각 구별로 승차 총 승객수, 하차 총 승객수 확인하기

- 실습문제 8. 각 구별로 승차 평균 승객수, 하차 평균 승객수 확인하기

- 실습문제 9. 데이터 합치기

In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 실습문제 1. 데이터 로딩하기

In [2]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
bus = pd.read_csv('BUS_STATION_BOARDING_MONTH_202301.csv', encoding ='cp949')

#### 실습문제 2. 기본정보 확인하기

In [3]:
# info 활용
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242090 entries, 0 to 1242089
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   사용일자        1242090 non-null  int64 
 1   노선번호        1242090 non-null  object
 2   노선명         1242090 non-null  object
 3   표준버스정류장ID   1242090 non-null  int64 
 4   버스정류장ARS번호  1242090 non-null  object
 5   역명          1242090 non-null  object
 6   승차총승객수      1242090 non-null  int64 
 7   하차총승객수      1242090 non-null  int64 
 8   등록일자        1242090 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 85.3+ MB


In [4]:
# Shape 확인
bus.shape

(1242090, 9)

In [5]:
# describe
bus.describe()

,사용일자,표준버스정류장ID,승차총승객수,하차총승객수,등록일자
count,1.242090e+06,1.242090e+06,1.242090e+06,1.242090e+06,1.242090e+06
mean,2.023012e+07,1.254219e+08,9.824638e+01,9.616134e+01,2.023013e+07
std,8.943980e+00,6.338575e+07,1.458214e+02,1.326740e+02,2.614456e+01
min,2.023010e+07,1.000000e+08,0.000000e+00,0.000000e+00,2.023010e+07
25%,2.023011e+07,1.079004e+08,1.200000e+01,1.500000e+01,2.023011e+07
50%,2.023012e+07,1.140002e+08,5.200000e+01,5.600000e+01,2.023012e+07
75%,2.023012e+07,1.210001e+08,1.290000e+02,1.280000e+02,2.023013e+07
max,2.023013e+07,9.998000e+08,4.414000e+03,4.022000e+03,2.023020e+07


#### 실습문제 3. 버스정류장 위치를 구 별로 구분하기

In [6]:
bus['자치구'] = bus['버스정류장ARS번호'].str.slice(start=0, stop=2)

In [7]:
bus['자치구'] = bus['자치구'].map({ '01': '종로구', '02': '중구', '03': '용산구', '04': '성동구', '05': '광진구', '06': '동대문구', '07': '중랑구', '08': '성북구', '09': '강북구', '10': '도봉구', '11': '노원구', '12': '은평구', '13': '서대문구', '14': '마포구', '15': '양천구', '16': '강서구', '17': '구로구', '18': '금천구', '19': '영등포구', '20': '동작구', '21': '관악구', '22': '서초구', '23': '강남구', '24': '송파구', '25': '강동구'})

#### 실습문제 4. 가상 버스정류장 제거하기

In [8]:
bus = bus.dropna() # 자치구가 없는 nan정류장
virtural_index = bus[bus['역명'].str.contains('가상')].index 
bus = bus.drop(virtural_index)

In [9]:
bus = bus.astype({"버스정류장ARS번호" :"int"})

## 서울 정류장 위치정보 import

In [11]:
stop = pd.read_csv('서울시 버스정류소 위치정보.csv', encoding = 'cp949')

In [13]:
stop.rename(columns={'정류소번호' : '버스정류장ARS번호'}, inplace = True)

In [14]:
df = pd.merge(bus,stop, how = 'inner', on = '버스정류장ARS번호')

In [15]:
df = df.drop(['X좌표','Y좌표','정류소명'],axis=1)

In [16]:
df['버스정류장ARS번호'].nunique()

10555

#### 실습문제 5. 구 별로 버스정류장이 몇 개나 있을까?

In [52]:
# bus_station_num = df.groupby(by = ['자치구'],as_index=False)[['버스정류장ARS번호']].nunique()
bus_station_NO = df.groupby(by = ['자치구'],as_index=False)[['버스정류장ARS번호']].nunique()
bus_station_NO.rename(columns = {'버스정류장ARS번호' : '버스정류장 개수' }, inplace = True)

bus_station_NO

,자치구,버스정류장 개수
0,강남구,496
1,강동구,360
2,강북구,412
3,강서구,566
4,관악구,465
5,광진구,270
6,구로구,481
7,금천구,346
8,노원구,514
9,도봉구,358


#### 실습문제 6. 구 별로 버스 노선이 몇 개나 지나갈까?

In [ ]:
# nunique !가 몬지 보기 ->
# 여기서는 중복되서 나타남???

In [53]:
bus_line_num = df.groupby(by = ['자치구'],as_index=False)[['노선번호']].nunique()
bus_line_num.rename(columns = {'노선번호' : '노선 개수' }, inplace = True)

bus_line_num 

,자치구,노선 개수
0,강남구,95
1,강동구,22
2,강북구,67
3,강서구,48
4,관악구,81
5,광진구,43
6,구로구,78
7,금천구,57
8,노원구,58
9,도봉구,47


#### 실습문제 7. 각 구 별로 승차 총 승객수, 하차 총 승객수 확인하기

In [60]:
bus_pass_num = df.groupby(by = ['자치구'],as_index=True)[['승차총승객수','하차총승객수']].sum()
bus_pass_num

,승차총승객수,하차총승객수
자치구,,
강남구,7605690,7218161
강동구,2769288,2702980
강북구,4619962,4476588
강서구,4599504,4573299
관악구,6780412,6924027
광진구,2672110,2654306
구로구,4846677,4647336
금천구,3724507,3540551
노원구,3973240,3922938


#### 실습문제 8. 각 구 별로 승차 평균 승객수, 하차 평균 승객수 확인하기

In [51]:
bus_pass_mean = df.groupby(by = ['자치구'],as_index=False)[['승차총승객수','하차총승객수']].mean()

bus_pass_mean.rename(columns = {'승차총승객수' : '평균 승차 승객수', '하차총승객수' : '평균 하차 승객수' }, inplace = True)

bus_pass_mean

,자치구,평균 승차 승객수,평균 하차 승객수
0,강남구,114.536624,108.700696
1,강동구,86.337896,84.270616
2,강북구,123.005458,119.188157
3,강서구,83.868276,83.390449
4,관악구,130.903566,133.676217
5,광진구,105.746567,105.041988
6,구로구,113.271875,108.613069
7,금천구,119.298751,113.406502
8,노원구,77.294374,76.315812
9,도봉구,98.687555,95.708339


#### 실습문제 9. 데이터 합치기

In [58]:
bb1 = pd.merge(bus_station_NO, bus_line_num, how='inner',on='자치구')
bb2 = pd.merge(bb1, bus_pass_num, how='inner',on='자치구')
bb3 = pd.merge(bb2, bus_pass_mean, how='inner',on='자치구')

In [59]:
bb3

,자치구,버스정류장 개수,노선 개수,승차총승객수,하차총승객수,평균 승차 승객수,평균 하차 승객수
0,강남구,496,95,7605690,7218161,114.536624,108.700696
1,강동구,360,22,2769288,2702980,86.337896,84.270616
2,강북구,412,67,4619962,4476588,123.005458,119.188157
3,강서구,566,48,4599504,4573299,83.868276,83.390449
4,관악구,465,81,6780412,6924027,130.903566,133.676217
5,광진구,270,43,2672110,2654306,105.746567,105.041988
6,구로구,481,78,4846677,4647336,113.271875,108.613069
7,금천구,346,57,3724507,3540551,119.298751,113.406502
8,노원구,514,58,3973240,3922938,77.294374,76.315812
9,도봉구,358,47,3152870,3057690,98.687555,95.708339
